# Ch9 이미지 기하학적 변환  
1) 9- 회전(p182)  
cv.getRotationMatrix2D(center, degree, scale)  
center : 중심점  예) (100,100)  
degree : 회전각도 예) 45 단위가 degree, radian이 아님  
scale : 이미지 배율\
\
 cv.getRotationMatrix2D를 사용해서 회전 행렬 M 만들기  
 cv.warpAffine 함수 사용해서 이미지에 회전 변환 적용

In [2]:
import cv2



img_color = cv2.imread('./9.1/cat.jpg')
cv2.imshow("color", img_color)


height, width = img_color.shape[:2]

 
M = cv2.getRotationMatrix2D((width/2.0, height/2.0), 45, 1)  


img_rotated = cv2.warpAffine(img_color, M, (width, height))

cv2.imshow("rotation", img_rotated)
cv2.waitKey(0)


cv2.destroyAllWindows()

2) 9-2 크기 조정  
dst = cv.resize(src,dsize, dst, fx, fy, interpolation)  
src : 원본이미지  
dst : 결과이미지  
dszie :  결과 이미지의 크기  
fx, fy : 수평, 수직방향 크기 조정 비율  
interpolation : 보간법(2가지가 있다.  INTER_LINEAR, INTER_CUBIC  
INTER_LINEAR가 상대적으로 빠르지만 선명도가 상대적으로 낮다)

In [7]:
import cv2 as cv


# 원본 이미지
img_color = cv.imread('./9.2/cat.jpg')
cv.imshow("original", img_color)

# 2배 이미지 desired size을 none으로 사양 안함 fx,fy로 확대 축소
img_result = cv.resize(img_color, None, fx=2, fy=2, 
    interpolation = cv.INTER_CUBIC)
cv.imshow("x2 INTER_CUBIC", img_result)

# 3배 이미지 dsize를 통해서 확대 축소, fx, fy는 사용x
height, width = img_color.shape[:2]
img_result = cv.resize(img_color, (3*width, 3*height), 
    interpolation = cv.INTER_LINEAR )
cv.imshow("x3 INTER_LINEAR", img_result)

# 1/2배 이미지 fx, fy로 확대 축소
img_result = cv.resize(img_color, None, fx=0.5, fy=0.5, interpolation = cv.INTER_AREA)
cv.imshow("x0.5 INTER_AREA", img_result)


cv.waitKey(0)
cv.destroyAllWindows()

3) 9-3 이동  
함수로 이미지를 직접적으로 이동하는것이 아님  
M <- 변환 행렬, 이동하는 변환행렬을 통해서 이동함  
cv.warpAffine(img, M, dsize)  
img : 적용할 이미지  
M : 변환행렬  
dsize : 출력이미지의 크기 (너비,높이)

In [2]:
import cv2 as cv
import numpy as np


img_color = cv.imread('./9.3/cat.jpg')
cv.imshow("original", img_color)


height, width = img_color.shape[:2]


M = np.float32([[1, 0, 100], [0, 1, 50]]) 

 
img_translation = cv.warpAffine(img_color, M, (width,height))

cv.imshow("translation", img_translation)


cv.waitKey(0)
cv.destroyAllWindows()

4) 9-4 아핀 변환\
\
입력 이미지의 세점 -> 출력이미지의 세점 간의 변환\
\
1.입력이미지의 세점, 출력이미지의 세점을 설정  
2.두 이미지간 대응하는 세 점을 안다면 getAffineTransform 아핀변환 행렬을 얻을 수 있음\
\
warpAffine 함수를 이용해서 이미지에 적용

In [3]:
import numpy as np
import cv2 as cv


point_list = []

def mouse_callback(event, x, y, flags, param):


    if event == cv.EVENT_LBUTTONDOWN:

        print("(%d, %d)" % (x, y))

        point_list.append((x, y))
        cv.circle(img_color, (x, y), 3, (0, 0, 255), -1)


 
cv.namedWindow('source')
cv.setMouseCallback('source', mouse_callback)


img_color = cv.imread('./9.4/test.png')




while(True):

    cv.imshow('source', img_color)


    if cv.waitKey(1) == 32: 
        break


height, weight = img_color.shape[:2]

# 1. 원본 3점과 원하는 점 3점을 설정한다.
pts1 = np.float32([point_list[0], point_list[1], point_list[2]])
pts2 = np.float32([point_list[0], point_list[1], point_list[2]])
pts2[1][1] += 100

# 세 점의 원본 좌표와 원하는 이미지의 좌표 3점을 넣으면 변환행렬 M이 구해진다.
M = cv.getAffineTransform(pts1,pts2)

#구해진 M 행렬을 warpAffine에 넣는다
img_result = cv.warpAffine(img_color, M, (weight,height))

 
cv.imshow("result", img_result)
cv.waitKey(0)
cv.destroyAllWindows()


(80, 110)
(248, 151)
(119, 231)


5) 9-5 퍼스펙티브 변환\
\
3차원 공간에 있는 물체를 2차원 평면에 올려 놓는다  
입력 4점, 대응되는 출력 4점이 필요하다\
\
1)입력 네점, 출력 네점을 설정  
2)M= cv.getAffineTransform(입력 네점, 출력 네점)\
\
변환 행렬 M을 계산\
\
3)cv.warpAffine 함수를 사용한다


In [10]:
import numpy as np
import cv2



src = np.zeros([4, 2], dtype=np.float32)
idx = 0

def mouse_callback(event, x, y, flags, param):
    global point_list, idx


    if event == cv2.EVENT_LBUTTONDOWN:

        src[idx] = (x, y)
        idx = idx + 1

        print("(%d, %d)" % (x, y))

        cv2.circle(img_color, (x, y), 10, (0, 0, 255), -1)



cv2.namedWindow('original')
cv2.setMouseCallback('original', mouse_callback)



img_color = cv2.imread('./9.5/test.jpg')
img_original = img_color.copy()


while(True):

    cv2.imshow("original", img_color)


    height, width = img_color.shape[:2]



    if cv2.waitKey(1) == 32: 
        break


dst = np.float32([[0,0],[width,0],[0,height],[width,height]])


M = cv2.getPerspectiveTransform(src,dst)


img_result = cv2.warpPerspective(img_original, M, (width,height))


cv2.imshow("result1", img_result)
cv2.waitKey(0)
cv2.destroyAllWindows()

(232, 59)
(372, 67)
(42, 319)
(581, 304)
